Note: this notebook is set up to run with the env.yml containing the name 'polaris_datasets'

# Dataset creation with [Polaris](https://github.com/polaris-hub/polaris)
The first step of creating a benchmark is to set up a standard dataset which allows accessing the curated dataset (which has been demonstrated in <01_ncats_solubility_data_curation.ipynb>), and all necessary information about the dataset such as data source, description of endpoints, units etc. 

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import pathlib

import pandas as pd
import datamol as dm
import numpy as np

from polaris.cli import PolarisHubClient

# polaris dataset
from polaris.dataset import Dataset, ColumnAnnotation
from polaris.dataset._column import Modality

from polaris.utils.types import HubOwner


root = pathlib.Path("__file__").absolute().parents[3]
os.chdir(root)
sys.path.insert(0, str(root))
from utils.docs_utils import load_readme

In [2]:
# Get the owner and organization
org = "polaris"
data_name = "ncats_adme/Solubility"
dirname = dm.fs.join(root, f"org-{org}", data_name)
gcp_root = f"gs://polaris-public/polaris-recipes/org-{org}/{data_name}"

owner = HubOwner(slug=org.lower(), type="organization")
owner

HubOwner(slug='polaris', external_id=None, type='organization')

In [3]:
BENCHMARK_DIR = f"{gcp_root}/benchmarks"
DATASET_DIR = f"{gcp_root}/datasets"
FIGURE_DIR =  f"{gcp_root}/figures"

## Load existing data

In [4]:
PATH = f'{gcp_root}/data/curation/{data_name}_curated.csv'
table = pd.read_csv(PATH)
table.columns

Index(['PUBCHEM_SID', 'SMILES', 'PUBCHEM_ACTIVITY_OUTCOME',
       'PUBCHEM_ACTIVITY_SCORE', 'PHENOTYPE', 'KINETIC_AQUEOUS_SOLUBILITY',
       'MOL_smiles', 'MOL_molhash_id', 'MOL_molhash_id_no_stereo',
       'MOL_num_stereoisomers', 'MOL_num_undefined_stereoisomers',
       'MOL_num_defined_stereo_center', 'MOL_num_undefined_stereo_center',
       'MOL_num_stereo_center', 'MOL_undefined_E_D', 'MOL_undefined_E/Z',
       'OUTLIER_PUBCHEM_ACTIVITY_OUTCOME',
       'OUTLIER_KINETIC_AQUEOUS_SOLUBILITY', 'AC_PUBCHEM_ACTIVITY_OUTCOME',
       'AC_KINETIC_AQUEOUS_SOLUBILITY'],
      dtype='object')

### Define `Dataset` object

In [5]:
version = "v1"
dataset_name = f"ncats_solubility-{version}"

### Below we specify the meta information of data columns

In [7]:
# Additional meta-data on the column level
annotations = {
     "MOL_molhash_id": ColumnAnnotation(
        description="Molecular hash ID. See <datamol.mol.hash_mol>"
    ),
    "MOL_smiles": ColumnAnnotation(
        description="Molecule SMILES string", modality="molecule"
    ),
    "PUBCHEM_ACTIVITY_OUTCOME": ColumnAnnotation(
        description="Binarized label based on the phenotype observed, active if Moderate/High Solubility (class = 1) or inactive if Low Solubility (class = 0).",
        user_attributes={"ref_col": "PHENOTYPE"},
    ),
    "PUBCHEM_ACTIVITY_SCORE": ColumnAnnotation(
        description="Whole number in solubility (ug/mL) of the compound.",
        user_attributes={"ref_col": "KINETIC AQUEOUS SOLUBILITY", "unit": "ug/mL"},
    ),
    "PHENOTYPE": ColumnAnnotation(
        description="Indicates type of activity observed: 0-10: Low Solubility (Class = 0); >10: Moderate/High Solubility (Class = 1)",
        user_attributes={"ref_col": "KINETIC AQUEOUS SOLUBILITY"},
    ),
    "KINETIC_AQUEOUS_SOLUBILITY": ColumnAnnotation(
        description="Numerical value of the observed aqueous solubility.",
        user_attributes={"unit": "ug/mL"},
    ),
}

### Define `Dataset` object

In [9]:
version = "v1"
dataset_name = f"ncats_solubility-{version}"

In [12]:
dataset = Dataset(
    # The table is the core data-structure required to construct a dataset
    table=table[annotations.keys()], 
    # Additional meta-data on the dataset level.
    name=dataset_name,
    description="ADME Solubility experiment data released by the National Center for Advancing Translational Sciences",
    source="https://doi.org/10.1038/s41598-020-77327-0",
    annotations=annotations,
    tags=["ADME"],
    license="CC-BY-4.0",
    user_attributes={"year": "2021"},
    curation_reference="https://github.com/polaris-hub/polaris-recipes/org-Polaris/ncats_adme/Solibility/01_ncats_solubility_data_curation.ipynb",
    owner=dict(user_id="polaris", slug="polaris"),
    readme=load_readme("org-Polaris/ncats_adme/Solubility/ncats_solubility_readme.md")
)

In [13]:
# save the dataset to GCP
SAVE_DIR = f"{DATASET_DIR}/{dataset_name}"
dataset_path = dataset.to_json(SAVE_DIR)
dataset_path

'gs://polaris-public/polaris-recipes/org-polaris/ncats_adme/Solubility/datasets/ncats_solubility-v1/dataset.json'

In [14]:
# upload to Polaris Hub
# dataset.upload_to_hub(owner=owner, access="private")